# Generate Segmentation Images based on NetSurfaceProblem

In [7]:
"""
To make sure, the visualization part works, start jupyter notebook with the command:
$ jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000
or change the data rate limit in the config file
"""

%matplotlib qt

import time
import copy
import numpy as np
from skimage.filters import gaussian

from tifffile import imread, imsave
import _pickle as pickle

import matplotlib.pyplot as plt

from spimagine import volfig, volshow
from spimagine import EllipsoidMesh, Mesh

%reload_ext autoreload
%autoreload 2

from Netsurface3d import NetSurf3d
from data4d import Data4d
from stitch_surfaces import StitchSurfaces
from netsurface3d_radial import NetSurf3dRadial
from data4d_radial import Data4dRadial
from Visualization import Visualization
from CrossSection import CrossSection

# Load and Initialize

In [8]:
#filenames of images and of mask have to be in same order

#filenames = [ '/Users/wissmann/BobSeg/FlySegCropped.tif']
#filenames = [ '/Users/wissmann/BobSeg/Cell_croppedModel-0.0.1-3D_Sim-PSF(0.2 0.2 0.6) PXS(0.1 0.1 0.1)-1.tif']
filenames = [ '/Users/wissmann/BobSeg/Vecad-flidsRed-48hpf.lif - fish1-48hpf-1.tif']
#filenames = [ '/Users/wissmann/BobSeg/s1e2_High.tif']
filenames_mask =[None,None,None]
#filenames_mask =np.array(['/Users/wissmann/BobSeg/tribo_mask.tif','/Users/wissmann/BobSeg/tribo_mask_y.tif','/Users/wissmann/BobSeg/tribo_mask_x.tif'])
#filenames_mask = np.array(['/Users/wissmann/BobSeg/fish_mask.tif','/Users/wissmann/BobSeg/fish_mask_y_end.tif','/Users/wissmann/BobSeg/fish_mask_x_end.tif'])
#filenames= ['/Users/wissmann/BobSeg/20180718_ecadgfp_0hAPF_disc05_cropped.tif']
#filenames_mask = np.array(['/Users/wissmann/BobSeg/wing_mask_z.tif','/Users/wissmann/BobSeg/wing_mask_y-2.tif',None])
#filenames = [ '/Users/wissmann/BobSeg/easy2surfaces.tif']
#filenames = ['/Users/wissmann/BobSeg/easypeasy2.tif']
#filenames_mask = [ '/Users/wissmann/BobSeg/easymask.tif']
#filenames = [ '/Users/wissmann/BobSeg/HyperStack.tif']
#filenames = [ '/Users/wissmann/BobSeg/HyperStack3.tif']
#filenames = [ '/Users/wissmann/BobSeg/debugdx.tif']
#filenames = ['/Users/wissmann/BobSeg/nonquadratictest.tif']

In [9]:
#Methods : [' z  ,  y ,  x , rad']
chosen  =  [  0  ,  0 ,  0 ,  1  ]

#if you change chosen, Kernel restart is required

In [10]:
# Parameters for Netsurf with Base Graph
#             [ z, y, x]
K           = [70,100,100]                 
max_delta_k = [5 , 4, 4]
dx          = [20, 20, 10]
dy          = [20, 10, 20]
surfaces    = [ 2, 2, 2]
min_dist    = [ 5, 5, 5]                 #In terms of K
max_dist    = [60,80,80]                 #In terms of K
plot_base_graph =False
#create column from.. to..
c_from      = [ 0, 0, 0] 
c_to        = [ 0, 0, 0]          #TODO (atm always -top- border of image)

In [22]:
# Parameters for Radial Netsurf
K_Rad            = 100
max_delta_k_rad  = 4
max_rs           = [(700,700,70),(700,700,200)]                                   #In px (x,y,z)
min_rs           = (1,1,1)                                         #In px (x,y,z)
centers          = [(500,500,140),(200,200,100)]                   #In px (x,y,z)

# Segmentation

In [23]:
data={}
Bg={}
Ra={}

#Base graph
for i in range(3):
    if chosen[i]==1:
        data[i]=Data4d(filenames, axis=i, filenames_mask=filenames_mask[i], pixelsize=(1.,1.,1.), silent=False, plane=None )
        data[i].set_seg_params(K[i],max_delta_k[i],dx[i],dy[i],surfaces[i],min_dist[i],max_dist[i])
        Bg[i] = data[i].init_object("Bg_%s"%i)
        data[i].add_object_at( Bg[i], c_from[i], c_to[i], frame=0, frame_to=0, segment_it=True, plot_base_graph=plot_base_graph )
    else: data[i]=None
        
#Radial
if chosen[3]==1:
    data[3]=Data4dRadial( filenames, pixelsize=(1.,1.,1.), silent=False )
    data[3].set_seg_params(K_Rad,max_delta_k_rad)
    for center in range(len(centers)):
        Ra[center]=data[3].init_object("Ra_%s"%center)
        data[3].add_object_at(Ra[center], min_rs, max_rs[center], frame=0, seed=centers[center], frame_to=0, segment_it=True )
else: data[3]=None
    
data[5]=Data4d(filenames, axis=0, filenames_mask=filenames_mask[0], pixelsize=(1.,1.,1.), silent=True, plane=None )
images=data[5].images

Dimensions of frame 0 :  (213, 738, 792)
Added appearance for "Ra_0" in frame 0 with seed coordinates [500. 500. 140.]
      Optimum energy:  24321.0
      Nodes in/out:  19468 31732
Added appearance for "Ra_1" in frame 0 with seed coordinates [200. 200. 100.]
      Optimum energy:  27305.0
      Nodes in/out:  11830 39370
filenames_mask None


# Visualization
    Colour code for both 3d visualization and 2d cross-section plots: 
        Base graph (z,y,x): (green,blue,gold)
        Radial            : red

In [24]:
"""
positions of base-graph meshes (i.e. surface meshes) with respect to the whole object:
   ! (only needed in case you want show_union=True) !
ou : outermost 'upper' surface (e.g. membrane), 
ol : outermost 'lower' surface,    
iu : inner 'upper'
il : inner 'lower'

'upper' and 'lower' refer to the respective direction of the base graph (e.g. y for the y-method)
"""

#methods        =       z.       y.     x
#mesh_positions = [['ou','ol'],['ou'],['ol']]
mesh_positions = [['ou','ol'],['ou','ol'],['ou','ol']]

In [25]:
vis = Visualization(data, images, mesh_positions)

{}


### 3D Visualization    
     
    show_single: shows all meshes from all chosen netsurf possibilities
    show_union: shows one union mesh of the single meshes in new window  TODO
    export: exports meshes as .obj files e.g. to be opened in blender
    stitch: in case of base graph with more than 1 surface, stitches the surfaces together to simulate volume

In [26]:
vis.show_frame(f=0, stackUnits=[1.,1.,1.], show_single=True, show_union=False, export=False, stitch=[False,False,False])

INFO:spimagine.gui.glwidget | saving frame as /Users/wissmann/BobSeg/fish_rad_new.png


### 2D Cross-Sections
    plane_normal: so far only axis-parallel vectors possible ( [1,0,0], [0,1,0], [0,0,1] )

In [27]:
from_slice = 750
to_slice   = 750
levels  = np.arange(from_slice, to_slice+1, step=1)
#Kernel restart required for new images

In [29]:
vis.show_sections(f=0,plane_orig=[0,0,0], plane_normal=[1,0,0], num_slices=levels, show_image=True) #in [x,y,z]

(1,)
()
(1, 1, 19, 2, 3)
(1, 19, 2, 3)
